In [1]:
import numpy as np
import re

In [2]:
def get_kinematic_matrices(matrix):
    print(matrix * np.array([1, -1, -1, 1])[:, np.newaxis])
    print(np.linalg.lstsq(matrix, np.eye(4), rcond=None)[0])

# Inverse kinematic

## Symbol calculations

In [3]:
import sympy
sympy.init_printing()

In [4]:
Rx, Ry = sympy.symbols("R_x, R_y")
rx, ry = sympy.symbols("r_x, r_y")
vx, vy, w = sympy.symbols("v_x, v_y, \omega")

In [5]:
V = sympy.Matrix([vx, vy, 0]) + sympy.Matrix([0, 0, w]).cross(sympy.Matrix([Rx, Ry, 0]))
V_m = sympy.Matrix(V[0:2]).dot(sympy.Matrix([rx, ry]))
V_m = sympy.expand(V_m)

In [6]:
V_m

Rₓ⋅\omega⋅r_y - R_y⋅\omega⋅rₓ + rₓ⋅vₓ + r_y⋅v_y

## Numerical calculations

In [7]:
Rx1 = 189.09 / 2
Ry1 = 169.09 / 2
a = np.pi / 4
rwheel = 36.9 / 2

rx1 = np.cos(a)
ry1 = np.sin(a)
angles = [np.pi / 4,  45 / 180 * np.pi, ]
Rxs = [Rx1, Rx1, -Rx1, -Rx1]
Rys = [Ry1, -Ry1, Ry1, -Ry1]
rxs = [rx1, -rx1, rx1, -rx1]
rys = [-ry1, -ry1, ry1, ry1]

In [8]:
angles

[0.7853981633974483, 0.7853981633974483]

In [7]:
forward_matrix = np.zeros((4, 3), dtype=np.float32)
for i in range(4):
    forward_matrix[i] = np.array([rxs[i], rys[i], Rxs[i] * rys[i] - Rys[i] * rxs[i]])
forward_matrix = forward_matrix / rwheel
forward_matrix[:, 0:2] = forward_matrix[:, 0:2] * 1000 

In [8]:
forward_matrix * np.array([1, -1, 1, -1])[:, np.newaxis]

array([[ 24.55231857, -24.55231857,  -5.77973843],
       [ 24.55231857,  24.55231857,   5.77973843],
       [ 24.55231857,  24.55231857,  -5.77973843],
       [ 24.55231857, -24.55231857,   5.77973843]])

### Forward kinematic

In [9]:
np.linalg.lstsq(forward_matrix, np.eye(4), rcond=None)

(array([[ 0.01018234, -0.01018234,  0.01018234, -0.01018234],
        [-0.01018234, -0.01018234,  0.01018234,  0.01018234],
        [-0.04325455, -0.04325455, -0.04325455, -0.04325455]]),
 array([0.25, 0.25, 0.25, 0.25]),
 3,
 array([49.10463715, 49.10463715, 11.55947685]))

## Different angles for big robot center in geom center

<img src="files/photo_big_robot_2.jpg">

In [10]:
Rx1 = 189.09 / 2
Ry1 = 169.09 / 2
Rx2 = 189.09 / 2
Ry2 = 169.09 / 2

Rxs = [-Rx1, -Rx2, Rx2, Rx1]
Rys = [Ry1, -Ry2, -Ry2, Ry1]
alphas = (np.array([180 - 45, 180 + 45, -45, 45]) - 90) * np.pi / 180
rxs = np.cos(alphas)
rys = np.sin(alphas)

In [11]:
rxs

array([ 0.70710678, -0.70710678, -0.70710678,  0.70710678])

In [12]:
rys

array([ 0.70710678,  0.70710678, -0.70710678, -0.70710678])

In [13]:
forward_matrix = np.zeros((4, 3), dtype=np.float32)
for i in range(4):
    forward_matrix[i] = np.array([rxs[i], rys[i], Rxs[i] * rys[i] - Rys[i] * rxs[i]])
forward_matrix = forward_matrix / rwheel
forward_matrix[:, 0:2] = forward_matrix[:, 0:2] * 1000 

In [16]:
forward_matrix * np.array([1, 1, 1, 1])[:, np.newaxis]

array([[ 38.32556534,  38.32556534,  -6.86372614],
       [-38.32556534,  38.32556534,  -6.86372614],
       [-38.32556534, -38.32556534,  -6.86372614],
       [ 38.32556534, -38.32556534,  -6.86372614]])

In [19]:
H = forward_matrix
(np.linalg.inv(H.T.dot(H))).dot(H.T)

array([[ 0.00652306, -0.00652306, -0.00652306,  0.00652306],
       [ 0.00652306,  0.00652306, -0.00652306, -0.00652306],
       [-0.03642336, -0.03642336, -0.03642336, -0.03642336]], dtype=float32)

# Calibration

### New big robot

In [16]:
y_exp = 890
y_true = 890 - 83 
x_exp = 1719
x_true = 1719 + 36
a_exp = np.pi
a_true = np.pi - 1 / 40

In [17]:
k = np.array([x_exp / x_true, y_exp / y_true, a_exp / a_true])

In [18]:
forward_matrix_1 = forward_matrix * k[np.newaxis, :]

In [19]:
forward_matrix_1 * np.array([1, -1, 1, -1])[:, np.newaxis]

array([[ 24.04868127,  27.07752606,  -5.85666637],
       [ 30.82350247, -16.18349293,   4.53981119],
       [-30.82350247, -16.18349293,  -4.53981119],
       [-24.04868127,  27.07752606,   5.85666637]])

In [20]:
np.linalg.lstsq(forward_matrix_1, np.eye(4), rcond=None)

(array([[ 0.00783538, -0.01010818, -0.01010818,  0.00783538],
        [ 0.01360546,  0.00813161, -0.00813161, -0.01360546],
        [-0.05319911, -0.04150626, -0.04150626, -0.05319911]]),
 array([0.13159783, 0.36840217, 0.36840217, 0.13159783]),
 3,
 array([55.28884046, 44.61160972, 10.47949063]))

# Calibration data

In [21]:
def cvt_local2global(local_point, sc_point):
    point = np.zeros(3)
    x, y, a = local_point
    X, Y, A = sc_point
    point[0] = x * np.cos(A) - y * np.sin(A) + X
    point[1] = x * np.sin(A) + y * np.cos(A) + Y
    point[2] = a + A
    return point


def cvt_global2local(global_point, sc_point):
    point = np.zeros(3)
    x, y, a = global_point
    X, Y, A = sc_point
    point[0] = x * np.cos(A) + y * np.sin(A) - X * np.cos(A) - Y * np.sin(A)
    point[1] = -x * np.sin(A) + y * np.cos(A) + X * np.sin(A) - Y * np.cos(A)
    point[2] = a - A
    return point

def get_movement(point):
    x, y, a = point
    R = np.sqrt(x ** 2 + y ** 2) / 2 / np.abs(np.sin(a / 2))
    S = R * a
    angle = np.arctan2(y, x) - a / 2
    return np.array([S * np.cos(angle), S * np.sin(angle), a])

In [22]:
import re

In [23]:
points = []
cmds = []
with open("calibration_data.txt", 'r') as file:
    for line in file:
        m = re.match('- Translation: \[(.*)\]', line[:-1]) 
        if m:        
            points.append(np.zeros(3))
            points[-1][0:2] = np.fromstring(m.group(1), sep=', ')[0:2]
        m = re.match('[ ]*in RPY \(radian\) \[(.*)\]', line[:-1]) 
        if m:
            points[-1][2] = np.fromstring(m.group(1), sep=', ')[2]
        m = re.match('(-?[\d]+.?[\d]* -?[\d]+.?[\d]* -?[\d]+.?[\d]*)', line[:-1])
        if m:
            cmds.append(np.fromstring(m.group(1), sep=' ', dtype=np.float32))

points_1 = np.array(points)
cmds = np.array(cmds)

In [24]:
points = np.array([cvt_global2local([0, 0, 0], p) for p in points_1])
true_movements = np.array([get_movement(cvt_global2local(p2, p1)) for p1, p2 in zip(
    points[:-1], np.roll(points, -1, axis=0)[:-1])])

In [25]:
true_movements.round(2)
for i in range(len(cmds) - 1):
    print((cmds[i] - true_movements[i]).round(2))

[ 0.02 -0.03 -0.02]
[ 0.01 -0.   -0.03]
[ 0.1  -1.68 -3.07]
[-1.63 -0.06  0.05]
[ 0.    0.01 -0.02]
[0.02 1.9  6.04]
[-0.04 -0.04 -0.49]
[ 0.04 -0.03 -0.41]
[-1.99 -0.01  0.02]
[ 0.02  0.03 -0.36]
[0.45 0.02 5.72]
[-0.01 -0.01 -0.57]
[ 0.02 -2.01  0.02]
[-0.8  -0.    0.01]
[-0.    0.    0.01]
[-0.  0. -0.]
[ 0. -0.  0.]
[-0.1   1.01 -0.05]
[0.12 0.02 0.01]
[ 0.01  0.01 -0.02]


In [43]:
errors = cmds[:-1] - true_movements
conds = (np.abs(errors[:, 0]) < 0.1) * (np.abs(errors[:, 1]) < 0.1) *\
    (np.abs(errors[:, 2]) < 0.05)
X = true_movements[conds]
Y = cmds[:-1][conds]

In [44]:
K = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)
K.round(2)

array([[ 1.01, -0.  , -0.02],
       [ 0.03,  0.97, -0.06],
       [-0.  ,  0.01,  0.99]])

In [45]:
forward_matrix2 = forward_matrix_1.dot(K.T)

In [46]:
forward_matrix2 * np.array([1, -1, -1, 1])[:, np.newaxis]

array([[ 24.4161859 ,  27.2213531 ,  -5.60133577],
       [ 31.09554175, -15.04963672,   4.30557195],
       [ 31.07573411,  16.22788419,   4.56069397],
       [ 24.44932721, -25.11085301,  -6.02819498]])

In [47]:
np.linalg.lstsq(forward_matrix2, np.eye(4), rcond=None)

(array([[ 0.00726103, -0.01030299, -0.00983144,  0.00805   ],
        [ 0.014515  ,  0.00874849, -0.00807217, -0.01362862],
        [-0.0528441 , -0.04170392, -0.04277514, -0.05463641]]),
 array([0.13159783, 0.36840217, 0.36840217, 0.13159783]),
 3,
 array([55.96902104, 43.07527339, 10.34349934]))

### With calibration but without not diagonal coefficients

In [52]:
K1 = K.copy()
K1[2, 0:2] = 0 
K1.round(2)

array([[ 1.01, -0.  , -0.02],
       [ 0.03,  0.97, -0.06],
       [ 0.  ,  0.  ,  0.99]])

In [59]:
get_kinematic_matrices(forward_matrix_1.dot(K1.T))

[[ 24.4161859   27.2213531   -5.77869177]
 [ 31.09554175 -15.04963672   4.47936896]
 [ 31.07573411  16.22788419   4.47936896]
 [ 24.44932721 -25.11085301  -5.77869177]]
[[ 0.00735153 -0.01023156 -0.00975818  0.00814357]
 [ 0.01408402  0.00840837 -0.00842103 -0.01407421]
 [-0.05287003 -0.04172438 -0.04279612 -0.05466322]]


In [ ]:
np.ndarray.to